In [1]:
! pip install spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/f9/19/b60e1ebf4985ee2b33d85705b89a5024942b65dad04dbdc3fb46f168b410/spacy-3.8.7-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/50/6c/01ded95ddce33811c9766cae4ce32e0a54288da1d909ee2bcaa6ed13b9f1/murmurhash-1.0.13-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtai

In [2]:
! python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
input_file = "input/mia's_story.txt"
with open(input_file,encoding="Utf-8") as f:
    text = f.read()
text

'The Lost Kitten\n\nOne sunny afternoon, Mia was playing in her backyard when she heard a tiny "meow." She looked under the bushes and saw a small, gray kitten with white paws. The kitten looked scared and hungry.\n\nMia’s Decision\nMia gently picked up the kitten and asked her mom, "Can we keep it?" Mom smiled and said, "First, we need to check if it belongs to someone." They made posters with the kitten’s picture and their phone number.\n\nThe Search\nFor two days, no one called. Mia named the kitten "Socks" because of its white paws. She gave Socks a cozy bed, a bowl of milk, and a red ball to play with.\n\nA Happy Surprise\nOn the third day, old Mr. Jenkins from down the street called. He said, "That’s my cat, Whiskers!" When Mia brought Socks—now Whiskers—to Mr. Jenkins, he explained that Whiskers had slipped out when the door was open.\n\nA New Friendship\nMr. Jenkins thanked Mia for taking care of Whiskers. He even invited her to visit anytime. Mia was sad but happy Whiskers was

In [ ]:
import spacy 
import re 
nlp = spacy.load('en_core_web_sm')

def preprocess(text):
    text = text.replace("\n"," ")
    text = re.sub(r'\s+', ' ', text).strip()
    doc = nlp(text)
    sents = []
    for sent in doc.sents:
        print(sent)
        for token in sent:
            if token.pos_ == 'NOUN':
                print(f"  NOUN token: {token.text}")
        noun_count = sum(1 for token in sent if token.pos_ == 'NOUN')
        if noun_count >= 2:
            sents.append(sent.text)
    return sents

sentences = preprocess(text)
sentences

['The Lost Kitten One sunny afternoon, Mia was playing in her backyard when she heard a tiny "meow."',
 'She looked under the bushes and saw a small, gray kitten with white paws.',
 'Mia’s Decision Mia gently picked up the kitten and asked her mom, "Can we keep it?"',
 'They made posters with the kitten’s picture and their phone number.',
 'The Search For two days, no one called.',
 'Mia named the kitten "Socks" because of its white paws.',
 'She gave Socks a cozy bed, a bowl of milk, and a red ball to play with.',
 'On the third day, old Mr. Jenkins from down the street called.',
 'He said, "That’s my cat, Whiskers!"',
 'When Mia brought Socks—now Whiskers—to Mr. Jenkins, he explained that Whiskers had slipped out when the door was open.',
 'A New Friendship Mr. Jenkins thanked Mia for taking care of Whiskers.',
 'The next week, Mom surprised Mia with a new kitten from the animal shelter!']

In [ ]:
import random
def select_random_sents(sentences,max_choice=5):
    random_sents = random.sample(sentences,max_choice)
    return random_sents

In [ ]:
sent = sentences[0]

for token in sent:
    (token.pos_=='NOUN')

False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False


In [12]:
select_random_sents(sentences)

[When Mia brought Socks—now Whiskers—to Mr. Jenkins, he explained that Whiskers had slipped out when the door was open.,
 The Search For two days, no one called.,
 The Lost Kitten One sunny afternoon, Mia was playing in her backyard when she heard a tiny "meow.",
 The next week, Mom surprised Mia with a new kitten from the animal shelter!,
 A Happy Surprise]